# Demo Fraud Detection

## Intro

In this notebook, we give a demo of how to use social network analysis for insurance fraud detection. 

The data for this analysis is taken from Kaggle https://www.kaggle.com/datasets/rohitrox/healthcare-provider-fraud-detection-analysis in order to ensure repreducibility. 

We are going to start with loading the necessary packages and own implementation of the methods. Afterwards, we inspect the data. With these insights, we construct the network on which we will do further analyses. 

In [ ]:
from HelperFunctions import load_network
import pickle as pkl
import pandas as pd
import numpy as np
import excecute
from HelperFunctions import to_bipartite, feature_engineering
from BiRank import *
from metapath2vec import *
from stellargraph import StellarGraph
import networkx as nx
import Metrics
import scipy as sp

from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

## Data Loading

There are two sorts of patients, both of which we will use. The Inpatient data consists of patients that were admitted to the hospital, while the Outpatient data covers patients that visited the hospital, but were not admitted in it. Addtional fraud labels are present in a seperate data set. There are only fraud labels for the health care providers. Hence, we are  going to look for suspicious providers, not claims. 

In [ ]:
Incoming = pd.read_csv("data/archive (1)/Train_Inpatientdata-1542865627584.csv")
Outgoing = pd.read_csv("data/archive (1)/Train_Outpatientdata-1542865627584.csv")
labels = pd.read_csv("data/archive (1)/Train-1542865627584.csv")

Below, we do a very quick inspection of the different tables, in order to have a sense of the naming of the different features, and the format in which the fraud labels are provided. 

In [ ]:
Incoming.head()

In [ ]:
Outgoing.head()

In [ ]:
labels.head()

All data on patients is put together in one data set. 

In [ ]:
AllClaims = [Incoming, Outgoing]
AllClaims = pd.concat(AllClaims)
AllClaims.head()

## Setting up the network

In order to be able to work with the network via the stellargraph package, the network data, i.e., the nodes and edges, need to be in a specific format. We need data frames for the nodes, where the nodes are the index of the data frame, and the name of this index is "ID". This is done below. 

Note that we are going to extract the data from the tables per node type. This makes it possible to take the heterogeneity of the different nodes into account. 

In [ ]:
bene_nodes = AllClaims[["BeneID"]].rename(columns={"BeneID":"ID"}).drop_duplicates().set_index("ID")
claim_nodes = AllClaims[["ClaimID"]].rename(columns={"ClaimID":"ID"}).drop_duplicates().set_index("ID")
provider_nodes = AllClaims[["Provider"]].rename(columns={"Provider":"ID"}).drop_duplicates().set_index("ID")
physician_nodes = AllClaims[["AttendingPhysician"]].rename(columns={"AttendingPhysician":"ID"}).drop_duplicates().set_index("ID")

For the edges of the network, we will just take them as presented in the original data, i.e., we connect the claims with the other nodes as given in the entry. Here, the columns need to be named "source" and "target" in order for it to work. 

In [ ]:
bene_claim = AllClaims[["BeneID", "ClaimID"]].rename(columns={"BeneID":"source", "ClaimID":"target"})
provider_claim = AllClaims[["Provider", "ClaimID"]].rename(columns={"Provider":"source", "ClaimID":"target"})
phys_claim = AllClaims[["AttendingPhysician", "ClaimID"]].rename(columns={"AttendingPhysician":"source", "ClaimID":"target"})
edges = pd.concat([bene_claim, provider_claim, phys_claim]).reset_index(drop=True)

With everything set up, we can easily construct the network with stellargraph. We will transform it also to networkx since it will make it easier to do some of the calculations. 

In [ ]:
HG = StellarGraph({"claim": claim_nodes, "beneficiary": bene_nodes, "provider": provider_nodes, "physician": physician_nodes}, edges)

In [ ]:
HG_nx = HG.to_networkx()

Below, we are going to inspect the structure of the network. 

In [ ]:
print(HG.info())

In [ ]:
degrees = HG.node_degrees()
bene_deg = pd.Series([degrees[x] for x in list(bene_nodes.index)])
claim_deg = pd.Series([degrees[x] for x in list(claim_nodes.index)])
provider_deg = pd.Series([degrees[x] for x in list(provider_nodes.index)])
physician_deg = pd.Series([degrees[x] for x in list(physician_nodes.index)])
perc = [.25,.50,.75,.85,.95, .99] 

In [ ]:
claim_deg.describe(percentiles = perc)

In [ ]:
bene_deg.describe(percentiles = perc)

In [ ]:
provider_deg.describe(percentiles = perc)

In [ ]:
physician_deg.describe(percentiles = perc)

In order to be able to incorportate the fraud labels, they need to be put into 0-1. Also, the providers need to be the index of the table, and the column with fraud labels for the algorithms needs to be called "Fraud".

In [ ]:
labels = labels.set_index("Provider")

In [ ]:
num_labels = [0 if x == "No" else 1 for x in labels["PotentialFraud"]]
labels["Fraud"] = num_labels

In [ ]:
labels

Since everything is now numerical, we can quickly see the relative number of fraudulent providers. Below, we see that is is around 9%.

In [ ]:
labels["Fraud"].mean()

With everything nicely set up, we can move to the calculation of the different network embeddings. 

## Network Embeddings

### Metapath2Vec

The first embedding algorithm we are going to consider is the metapath2vec method. This method walks around in the network in order to form random paths, which can be converted into an embedding using NLP methods. Here, we restrict the possible paths to a couple of metapath.

Below, one can see that each path needs to start and stop in a provider node, and that we do not allow long detours in order to get there. 

These metapaths are fed into the method, which is already implementede in its dedicated python file. 

In [ ]:
metapaths = [
            ["provider", "claim", "provider"],
            ["provider", "claim","physician", "claim", "provider"],
            ["provider", "claim","beneficiary", "claim", "provider"]
            ]

In [ ]:
node_ids, node_embeddings, node_targets = Metapath2vec(HG,
                                                       metapaths, 
                                                       dimensions = 64,
                                                       num_walks = 2,
                                                       walk_length = 7,
                                                       context_window_size = 5)

With the embeddings constructed, we need to link the right embedding with the corresponding provider and hence with the corresponding fraud label. 

In [ ]:
HG.nodes()

In [ ]:
indices = [i for i in range(len(node_targets)) if node_targets[i]=="provider" ]
providers_sorted = [node_ids[i] for i in range(len(node_targets)) if node_targets[i]=="provider" ]

These embeddings are used to construct a fraud detection model. For this, we are going to use a gradient boosting classifier as implemented in scikit-learn. 

In [ ]:
X = node_embeddings[indices]
X = pd.DataFrame(X).sort_index()
y = labels.sort_index().loc[providers_sorted]["Fraud"]

In [ ]:
train_size = int(round(0.6 * len(y), 0)); train_size

In [ ]:
X_train = X.iloc[:train_size,:]
y_train = y[:train_size]

X_test = X.iloc[train_size:,:]
y_test = y[train_size:]

In [ ]:
y_test.sum()

In [ ]:
embedding_model = GradientBoostingClassifier(n_estimators=100,
                                                 max_depth=2).fit(X_train, y_train)

y_pred_meta = embedding_model.predict_proba(X_test)[:, 1]

The quality of the model is analysed using the AUC, the average precision and the lift curve. 

In [ ]:
fpr_meta, tpr_meta, thresholds = metrics.roc_curve(y_test, y_pred_meta)
plt.plot(fpr_meta, tpr_meta)
plt.plot([0, 1], [0, 1], color="grey", alpha=0.5)
plt.title("AUC-ROC: " + str(np.round(metrics.auc(fpr_meta, tpr_meta), 3)))

In [ ]:
precision_meta, recall_meta, thresholds = metrics.precision_recall_curve(y_test,y_pred_meta)
AP_meta = np.round(metrics.average_precision_score(y_test, y_pred_meta),3)
plt.plot(recall_meta,precision_meta, label =str(AP_meta), alpha =0.7)
plt.title("AUC-PRC")
plt.legend()

In [ ]:
steps = np.arange(start = 0.05, stop = 1, step = 0.001)
lft_meta = Metrics.lift_curve_values(y_test, y_pred_meta, steps)
plt.plot(steps, lft_meta, alpha = 0.7)

plt.title("Lift Curve")


### BiRank 

The next method is BiRank. Here, we need to reconstruct the network into a bipartite one. We do this by connecting the providers (group 1) to the corresponding physicians and beneficiaries (together group 2). We say that an edge is present between a provider and a physician/beneficiary, if the are part of the same claim. 

We need to do some simple matrix multiplications in order to obtain the correct adjacency matrix. 

In [ ]:
HG_claims = HG.nodes("claim")
HG_physicians = HG.nodes("physician")
HG_providers = providers_sorted#HG.nodes("provider")
HG_beneficiaries = HG.nodes("beneficiary")

In [ ]:
adj_C_Ph = HG.to_adjacency_matrix(
    list(HG_claims) + list(HG_physicians)
)[ :len(HG_claims), len(HG_claims): ]

adj_C_B = HG.to_adjacency_matrix(
    list(HG_claims) + list(HG_beneficiaries)
)[ :len(HG_claims), len(HG_claims): ]

adj_C_Pr = HG.to_adjacency_matrix(
    list(HG_claims) + list(HG_providers)
)[ :len(HG_claims), len(HG_claims): ]

In [ ]:
adj_Pr_B = adj_C_Pr.transpose() @ adj_C_B
adj_Pr_Ph = adj_C_Pr.transpose() @ adj_C_Pr

adjmat_bipartite = sp.sparse.hstack((adj_Pr_B, adj_Pr_Ph))

This new bipartite adjacency matrix will now be used for the calculation of the fraud scores for the medical providers. 

The BiRank method will be applied twice in order to take the temporal aspect into account. 

In [ ]:
fraud = {"FraudInd": labels["Fraud"][HG_providers].values}
fraudMat = pd.DataFrame(fraud)

provider_nodes = pd.DataFrame({"ID": HG_providers}).set_index("ID")

HG_parties = np.concatenate((HG_beneficiaries, HG_physicians))
party_nodes = pd.DataFrame({"ID": HG_parties}).set_index("ID")

ADJ = adjmat_bipartite.transpose().tocsr()

Claims_res, Parties_res, aMat, iterations, convergence = BiRank(ADJ, provider_nodes, party_nodes, fraudMat)

In [ ]:
ADJ = adjmat_bipartite.tocsr()
number_providers = ADJ.shape[0]
train_set_size = int(np.round(0.6*number_providers))
test_set_size = number_providers-train_set_size
ADJ_train= ADJ[:train_set_size,:]
fraudMat_train = fraudMat.iloc[:train_set_size]
provider_train = provider_nodes[:train_set_size]

ADJ = ADJ_train.transpose().tocsr()
Claims_res, Parties_res, aMat, iterations, convergence = BiRank(ADJ, provider_train, party_nodes, fraudMat_train)

fraud_train_res = {"FraudInd": Claims_res.sort_values("ID")["ScaledScore"].values}
test_set_fraud = {"FraudInd": [0]*test_set_size}
fraudMat_train_res = pd.DataFrame(fraud_train_res)
fraudMat_test_set = pd.DataFrame(test_set_fraud)
fraudMat_test = fraudMat_train_res.append(fraudMat_test_set)

ADJ = adjmat_bipartite.transpose().tocsr()
Claims_res, Parties_res, aMat, iterations, convergence = BiRank(ADJ, provider_nodes, party_nodes, fraudMat_test)

As before, we look at the models performance using the AUC, AP and lift. 

In [ ]:
y_test = fraudMat.iloc[train_set_size:]["FraudInd"]
y_pred_bi = Claims_res.sort_values("ID")[train_set_size:].ScaledScore
fpr_bi, tpr_bi, thresholds = metrics.roc_curve(y_test,y_pred_bi)

plt.plot(fpr_bi,tpr_bi)

plt.plot([0,1], [0,1], color = "grey", alpha = 0.5)
plt.title("AUC-ROC: "+str(np.round(metrics.auc(fpr_bi, tpr_bi),3)))

In [ ]:
precision_bi, recall_bi, thresholds = metrics.precision_recall_curve(y_test,y_pred_bi)
AP_bi = np.round(metrics.average_precision_score(y_test, y_pred_bi),3)
plt.plot(recall_bi,precision_bi, label =str(AP_bi), alpha =0.7)
plt.title("AUC-PRC")
plt.legend()

In [ ]:
steps = np.arange(start = 0.05, stop = 1, step = 0.001)
lft_bi = Metrics.lift_curve_values(y_test, y_pred_bi, steps)
plt.plot(steps, lft_bi, alpha = 0.7)

plt.title("Lift Curve")

### Traditional Network Features

The final set of features we are going to look at, are the more traditional ones. It consists of the degree and the betweenness centrality. These are also much easier to interprete than the other ones. 

We have run the code (now in text) before, and saved the results in an excel file. This saves us time when rerunning the notebook. 

As always, the model performance is analysed. 

In [ ]:
#from HelperFunctions import geodesic

In [ ]:
#full_geo_G = geodesic(HG_nx)

In [ ]:
deg_cen = nx.degree_centrality(HG_nx)
df_degcen = pd.DataFrame({'provider': [provider for provider in provider_nodes.index],
                          'degree': [deg_cen[provider] for provider in provider_nodes.index] })
df_degcen.head()

from HelperFunctions import load_network, feature_engineering
import networkx as nx
import networkit
import pandas as pd

nx_graph_1 = HG.to_networkx()
nx_graph_2 = nx.Graph(nx_graph_1)
G_nit = networkit.nxadapter.nx2nk(nx_graph_2)

zipped_nodes = zip(nx_graph_2.nodes(), range(nx_graph_2.number_of_nodes()))
node_keys = pd.DataFrame(zipped_nodes)

cl_cen = networkit.centrality.ApproxCloseness(G_nit, 10000).run().ranking()
cl_cen_df = pd.DataFrame(cl_cen)

cl_cen_nodes = node_keys.merge(cl_cen_df, left_on = 1, right_on = 0)

cl_cen_nodes = cl_cen_nodes[["0_x", "1_y"]]
cl_cen_nodes.columns= ["node_id", "Closeness Centrality"]

btw_cen = networkit.centrality.EstimateBetweenness(G_nit, 10000).run().ranking()

btw_cen_df = pd.DataFrame(btw_cen)
btw_cen_nodes = node_keys.merge(btw_cen_df, left_on = 1, right_on = 0)
btw_cen_nodes = btw_cen_nodes[["0_x", "1_y"]]
btw_cen_nodes.columns= ["node_id", "Betweenness Centrality"]

centralities = cl_cen_nodes.merge(btw_cen_nodes, on = "node_id")
centralities.to_csv("Centralities_medical.csv", index = False)

In [ ]:
Centralities_medical = pd.read_csv("Centralities_medical.csv").set_index("node_id")
Centralities_medical = Centralities_medical.loc[labels.index]

deg_cen = nx.degree_centrality(HG_nx)
df_degcen = pd.DataFrame({'provider': [provider for provider in provider_nodes.index],
                          'degree': [deg_cen[provider] for provider in provider_nodes.index] }).set_index("provider").loc[labels.index]

X = pd.merge(Centralities_medical, df_degcen, left_index = True, right_index = True).dropna().sort_index()

train_size = int(round(0.6 * len(labels), 0))

y = labels.sort_index()["Fraud"][X.index]

X_train = X.iloc[:train_size,:]
y_train = y[:train_size]

X_test = X.iloc[train_size:,:]
y_test = y[train_size:]

embedding_model = GradientBoostingClassifier(n_estimators=500,
                                                 subsample=0.8,
                                                 max_depth=3).fit(X_train, y_train)

y_pred_cen = embedding_model.predict_proba(X_test)[:, 1]

In [ ]:
fpr_cen, tpr_cen, thresholds = metrics.roc_curve(y_test, y_pred_cen)
plt.plot(fpr_cen, tpr_cen)
plt.plot([0, 1], [0, 1], color="grey", alpha=0.5)
plt.title("AUC-ROC: " + str(np.round(metrics.auc(fpr_cen, tpr_cen), 3)))

In [ ]:
precision_cen, recall_cen, thresholds = metrics.precision_recall_curve(y_test,y_pred_cen)
AP_cen = np.round(metrics.average_precision_score(y_test, y_pred_cen),3)
plt.plot(recall_cen,precision_cen, label =str(AP_cen), alpha =0.7)
plt.title("AUC-PRC")
plt.legend()

In [ ]:
steps = np.arange(start = 0.05, stop = 1, step = 0.001)
lft_cen = Metrics.lift_curve_values(y_test, y_pred_cen, steps)
plt.plot(steps, lft_cen, alpha = 0.7)

plt.title("Lift Curve")

### GraphSAGE

In [ ]:
import stellargraph as sg
import tensorflow as tf
import networkx as nx

from stellargraph.mapper import FullBatchNodeGenerator

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt

from stellargraph.layer import HinSAGE    
from stellargraph.mapper import HinSAGENodeGenerator, NodeSequence
from keras import layers
from tensorflow.keras import layers, optimizers, Model
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
import pandas as pd

We are going to re-initialise the data set, to be sure that no data problems will arise down the line

In [ ]:
Incoming = pd.read_csv("data/archive (1)/Train_Inpatientdata-1542865627584.csv")
Outgoing = pd.read_csv("data/archive (1)/Train_Outpatientdata-1542865627584.csv")
ben_data = pd.read_csv("data/archive (1)/Train_Beneficiarydata-1542865627584.csv")
labels = pd.read_csv("data/archive (1)/Train-1542865627584.csv")

AllClaims = [Incoming, Outgoing]
AllClaims = pd.concat(AllClaims)

Using the data, we can easily construct the network, consisting of nodes and edges. 

In [ ]:
bene_nodes = AllClaims[["BeneID"]].rename(columns={"BeneID":"ID"}).drop_duplicates().set_index("ID")
claim_nodes = AllClaims[["ClaimID"]].rename(columns={"ClaimID":"ID"}).drop_duplicates().set_index("ID")
provider_nodes = AllClaims[["Provider"]].rename(columns={"Provider":"ID"}).drop_duplicates().set_index("ID")
physician_nodes = AllClaims[["AttendingPhysician"]].rename(columns={"AttendingPhysician":"ID"}).drop_duplicates().set_index("ID")

nodes = {"beneficiary": bene_nodes, "claim": claim_nodes, "provider": provider_nodes, "physician": physician_nodes}

edges = [
    zip(AllClaims.BeneID, AllClaims.ClaimID),
    zip(AllClaims.Provider, AllClaims.ClaimID),
    zip(AllClaims.AttendingPhysician, AllClaims.ClaimID)
]

For the previous methods, we did not have additional feature information for the health care providers. Now, with GraphSAGE, we will incorporate the features of the other kinds of nodes in the network. 

Especially the patients have a lot of data being kept with their dosier. In addition, we incorporate two features for the claim data as well. 

In [ ]:
beneficiary_data = ben_data.set_index("BeneID")
beneficiary_data["Deceased"] = [0 if x else 1 for x in beneficiary_data["DOD"].isna()]
beneficiary_data["RenalDiseaseIndicator_01"] = [0 if x == 0 else 1 for x in ben_data["RenalDiseaseIndicator"]]
beneficiary_data["Gender_01"] = beneficiary_data["Gender"]-1
beneficiary_data["ChronicCond_Alzheimer_01"] = beneficiary_data["ChronicCond_Alzheimer"]-1
beneficiary_data["ChronicCond_Heartfailure_01"] = beneficiary_data["ChronicCond_Heartfailure"]-1
beneficiary_data["ChronicCond_KidneyDisease_01"] = beneficiary_data["ChronicCond_KidneyDisease"]-1
beneficiary_data["ChronicCond_Cancer_01"] = beneficiary_data["ChronicCond_Cancer"]-1
beneficiary_data["ChronicCond_ObstrPulmonary_01"] = beneficiary_data["ChronicCond_ObstrPulmonary"]-1
beneficiary_data["ChronicCond_Depression_01"] = beneficiary_data["ChronicCond_Depression"]-1
beneficiary_data["ChronicCond_Diabetes_01"] = beneficiary_data["ChronicCond_Diabetes"]-1
beneficiary_data["ChronicCond_IschemicHeart_01"] = beneficiary_data["ChronicCond_IschemicHeart"]-1
beneficiary_data["ChronicCond_Osteoporasis_01"] = beneficiary_data["ChronicCond_Osteoporasis"]-1
beneficiary_data["ChronicCond_rheumatoidarthritis_01"] = beneficiary_data["ChronicCond_rheumatoidarthritis"]-1
beneficiary_data["ChronicCond_stroke_01"] = beneficiary_data["ChronicCond_stroke"]-1

selected_features = [
    #"DOB",
    "Deceased",
    "Race",
    "RenalDiseaseIndicator_01",
    "State",
    "Gender_01",
    "ChronicCond_Alzheimer_01",
    "ChronicCond_Heartfailure_01",
    "ChronicCond_KidneyDisease_01",
    "ChronicCond_Cancer_01",
    "ChronicCond_ObstrPulmonary_01",
    "ChronicCond_Depression_01",
    "ChronicCond_Diabetes_01",
    "ChronicCond_IschemicHeart_01",
    "ChronicCond_Osteoporasis_01",
    "ChronicCond_rheumatoidarthritis_01",
    "ChronicCond_stroke_01"
]

beneficiary_data["DOB"] = pd.to_datetime(beneficiary_data["DOB"])

beneficiary_data = beneficiary_data[selected_features]
beneficiary_data.head()

In [ ]:
AllClaims["DurationStay"] =pd.to_numeric(pd.to_datetime(AllClaims["ClaimEndDt"])-pd.to_datetime(AllClaims["ClaimStartDt"]))/86400000000000

claim_data = AllClaims.set_index("ClaimID")

selected_features = [
    "DurationStay",
    "InscClaimAmtReimbursed"
]

claim_data["ClaimStartDt"] = pd.to_datetime(claim_data["ClaimStartDt"])

claim_data = claim_data[selected_features]
claim_data.head()

The GraphSAGE implementation in StellarGraph requires to have features present for all nodes. However, both the providers and physicians do not have additional information. Therefore, we set their features equal to 1.

In [ ]:
provider_data = provider_nodes.copy(deep = False)
provider_data["Feature"] = 1 

physician_data = physician_nodes.copy(deep = False)
physician_data["Feature"] = 1

In [ ]:
features = {"beneficiary": beneficiary_data, "claim": claim_data, "provider": provider_data, "physician": physician_data}

In [ ]:
class GraphConstruction:
    
    """
    This class initializes a networkX graph
    
     Parameters
    ----------
    nodes : dict(str, iterable)
        A dictionary with keys representing the node type, values representing
        an iterable container of nodes (list, dict, set etc.)
    edges : 2-tuples (u,v) or 3-tuples (u,v,d)
        Each edge given in the container will be added to the graph. 
    features: dict(str, (str/dict/list/Dataframe)
        A dictionary with keys representing node type, values representing the node
        data.      
    
    """
    
    g_nx = None
    node_features = None
    
    def __init__(self, nodes, edges, features = None):
        self.g_nx = nx.Graph()
        self.add_nodes(nodes)
        self.add_edges(edges)
        
        if features is not None:
            self.node_features = features
    
    def add_nodes(self, nodes):
        
        for key, values in nodes.items():
            self.g_nx.add_nodes_from(list(values.index), ntype=key)   
            
    def add_edges(self, edges):
        
        for edge in edges:
            self.g_nx.add_edges_from(edge)
            
    def get_stellargraph(self):
        return sg.StellarGraph.from_networkx(self.g_nx, node_type_attr="ntype", node_features=self.node_features) #sg.StellarGraph.from_networkx(self.g_nx) 
    
    def get_edgelist(self):
        edgelist = []
        for edge in nx.generate_edgelist(self.g_nx):
            edgelist.append(str(edge).strip('{}'))
        el = []
        for edge in edgelist:
            splitted = edge.split()
            numeric = map(float,splitted)
            el.append(list(numeric))
        return el

In [ ]:
graph = GraphConstruction(nodes, edges, features)
S = graph.get_stellargraph()
print(S.info())

In [ ]:
labels["Fraud"] = [0 if x == "No" else 1 for x in labels["PotentialFraud"]]
labels
labels_indexed = labels.set_index("Provider")[["Fraud"]]

train_size = int(round(0.5 * len(y), 0))
val_test_size = int(round(0.6 * len(y), 0)) - train_size

In [ ]:
train_subjects, test_subjects = train_test_split(
    labels_indexed, train_size=train_size, test_size=None, stratify=labels_indexed
)
val_subjects, test_subjects = train_test_split(
    test_subjects, train_size=val_test_size, test_size=None, stratify=test_subjects
)

In [ ]:
batch_size = 50
epochs = 10

num_samples = [2,32]
embedding_node_type = "provider"

In [ ]:
generator = HinSAGENodeGenerator(S, batch_size, num_samples, head_node_type = embedding_node_type)
train_gen = generator.flow(train_subjects.index, train_subjects["Fraud"])
val_gen = generator.flow(val_subjects.index, val_subjects["Fraud"])

model = HinSAGE(layer_sizes = [64, 64], generator = generator, dropout=0)
x_inp, x_out = model.build() 
prediction = layers.Dense(units=1, activation="sigmoid", dtype='float32')(x_out)
model = Model(inputs = x_inp, outputs=prediction)

model.compile(
    optimizer = optimizers.Adam(lr=1e-3),
    loss=losses.binary_crossentropy,
    metrics=["AUC"],
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es_callback = EarlyStopping(monitor="val_auc", patience=5, restore_best_weights=True)

import sklearn
# Calculate the weights for each class so that we can balance the data
weights = sklearn.utils.class_weight.compute_class_weight('balanced',
                                            classes = np.unique(labels["Fraud"]),
                                            y = labels["Fraud"].values)

class_weight = {0: weights[0], 1: weights[1]}

In [ ]:
history = model.fit(
    train_gen, 
    epochs = 50, 
    validation_data = val_gen,
    shuffle=False, # this should be False, since shuffling data means shuffling the whole graph
    verbose = 2,
    callbacks=[es_callback],
    class_weight=class_weight,
)

In [ ]:
test_gen =generator.flow(test_subjects.index, test_subjects["Fraud"])

test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
test_predictions = model.predict(test_gen)

In [ ]:
df = pd.DataFrame({
    "Subject": test_subjects.index, 
    "Label": test_subjects["Fraud"],
    "Predicted": test_predictions.squeeze()
})

y_test = df["Label"]
y_pred_sage = df["Predicted"]

In [ ]:
fpr_sage, tpr_sage, thresholds = sklearn.metrics.roc_curve(y_test, y_pred_sage)
plt.plot(fpr_sage, tpr_sage)
plt.plot([0,1],[0,1], color = "grey", alpha =0.5)
plt.title("AUC-ROC: " + str(np.round(sklearn.metrics.auc(fpr_sage, tpr_sage), 3)))

In [ ]:
precision_sage, recall_sage, thresholds = sklearn.metrics.precision_recall_curve(y_test, y_pred_sage)
AP_sage = np.round(sklearn.metrics.average_precision_score(y_test, y_pred_sage), 3)
plt.plot(recall_sage, precision_sage, label = str(AP_sage), alpha =0.7)
plt.title("AUC-PRC")
plt.legend()

In [ ]:
steps = np.arange(start = 0.05, stop = 1, step = 0.001)
lft_sage = Metrics.lift_curve_values(y_test, y_pred_sage, steps)
plt.plot(steps, lft_cen, alpha = 0.7)
plt.title("Lift Curve")

### Putting Everything Together

This final part of the demo puts all results together. In this way, we can use the different performance metric to compare the models with each other. 

In [ ]:
plt.plot(fpr_cen, tpr_cen, alpha = 0.7, label =  "Simple Network Features: " + str(np.round(sklearn.metrics.auc(fpr_cen, tpr_cen), 3)))
plt.plot(fpr_bi, tpr_bi, alpha = 0.7, label =  "BiRank: " + str(np.round(sklearn.metrics.auc(fpr_bi, tpr_bi), 3)))
plt.plot(fpr_meta, tpr_meta, alpha = 0.7, label =  "Metapath2Vec: " + str(np.round(sklearn.metrics.auc(fpr_meta, tpr_meta), 3)))
plt.plot(fpr_sage, tpr_sage, alpha = 0.7, label =  "GraphSAGE: " + str(np.round(sklearn.metrics.auc(fpr_sage, tpr_sage), 3)))
plt.plot([0, 1], [0, 1], color="grey", alpha=0.5)
plt.title("AUC-ROC")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.savefig("AUC_health.pdf")

In [ ]:
plt.plot(recall_cen,precision_cen, alpha = 0.7, label =  "Simple Network Features: " + str(AP_cen))
plt.plot(recall_bi,precision_bi, alpha = 0.7, label =  "BiRank: " + str(AP_bi))
plt.plot(recall_meta,precision_meta, alpha =0.7, label =  "Metapath2Vec: " + str(AP_meta))
plt.plot(recall_sage,precision_sage, alpha =0.7, label =  "GraphSAGE: " + str(AP_sage))
plt.title("AUC-PRC")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.savefig("AP_health.pdf")

In [ ]:
plt.plot(steps,lft_cen, alpha = 0.7, label =  "Simple Network Features")
plt.plot(steps,lft_bi, alpha = 0.7, label =  "BiRank")
plt.plot(steps,lft_meta, alpha =0.7, label =  "Metapath2Vec")
plt.plot(steps,lft_sage, alpha =0.7, label =  "GraphSAGE")
plt.title("Lift Curve")
plt.xlabel("p")
plt.ylabel("Lift")
plt.legend()
plt.savefig("Lift_health.pdf")

From these figures, we can conclude that the simple network features seem to be competitive and sometimes outperform the models based on more complex, and less interpretable, network embeddings. 